### Import python library

In [3]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

### 1. Read data: augment_24group_1620.csv필요

In [4]:
timestep = 3

In [5]:
PCT_lstm_train_X = pd.read_csv("PCT_lstm_final_train_X33.csv")
PCT_lstm_train_y = pd.read_csv("PCT_lstm_final_train_y33.csv")

PCT_lstm_test_X = pd.read_csv("PCT_lstm_final_test_X33.csv")
PCT_lstm_test_y = pd.read_csv("PCT_lstm_final_test_y33.csv")

In [6]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [7]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

#### 4) input shape로 변경 (row, timestep=2, feature)

ex) 
timestep = 2

* X_train_v 구성예시: [[1 ~ 24경기 데이터, 25 ~ 48경기 데이터], [49 ~ 72경기 데이터, 73 ~ 96경기 데이터] ]  
X_train_v.shape >> (2,2*x)             # x: 각 24group에 대한 변수 개수
* y_train_v 구성예시: 97 ~ 120 경기 승률

=> reshape

* X_train_v.shape >> (2,2,x)  # row, timestep, feature

### 2. 모델 구성(LSTM)
- optimizer: Adam -> lr(learning rate) 조절
- LSTM: 모델이 계속 동일한 결과값이 나올 때, input 뉴런 개수를 늘려야 한다는 글을 읽고 계속 input 노드 개수를 바꿔주면서 모델 생성중
- loss: MSE

- early_stop: patience를 크게하면 과적합 되는 경우가 있어서 최대한 작게 설정해둠
- batch_size: 모델이 계속 동일한 결과값이 나올 때, 데이터가 적어 batch size를 줄여보라는 글을 읽고 1로 설정해둠

In [8]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([])

idx = 0
for y in year:
    tmp1 = PCT_lstm_train_X[PCT_lstm_train_X["YEAR"] == y]
    tmp2 = PCT_lstm_train_y[PCT_lstm_train_y["YEAR"] == y]
    tmp3 = PCT_lstm_test_X[PCT_lstm_test_X["YEAR"] == y]
    tmp4 = PCT_lstm_test_y[PCT_lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train_o = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1).reset_index(drop=True)
        y_train_o = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1).reset_index(drop=True)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        y_pred_lst = []
        rms_lst = []
        
        for k in range(len(X_train_o)):
            rename = '{}_valid{}'.format(name,k)
            print(k)
            X_valid = X_train_o.loc[k,:]
            y_valid = y_train_o.loc[k,:]
            
            X_train = X_train_o.drop(k,0)
            y_train = y_train_o.drop(k,0)

            
            X_train_v = X_train.values
            y_train_v = y_train.values
            
            X_valid_v = X_valid.values
            y_valid_v = y_valid.values            

            X_test_v = X_test.values
            y_test_v = y_test.values

            X_train_t = X_train_v.reshape(X_train_v.shape[0], timestep,X_train_v.shape[1]//timestep)
            X_valid_t = X_valid_v.reshape(1, timestep,X_valid_v.shape[0]//timestep)
            X_test_t = X_test_v.reshape(X_test_v.shape[0], timestep,X_test_v.shape[1]//timestep)

            ## model
            K.clear_session() 

            model = Sequential()
            optimizer = Adam(lr=0.01)

            model.add(LSTM(100,input_shape = (timestep,X_train_v.shape[1]//timestep))) # (timestep, feature)
            model.add(Dense(1)) # output = 1
            model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

            model.summary()

            early_stop = EarlyStopping(monitor='val_loss', mode = 'min',patience=2, verbose=1)

            hist1 = model.fit(X_train_t, y_train_v, epochs=100,
                      batch_size=1, verbose=1, callbacks=[early_stop],validation_data = (X_valid_t,y_valid_v))
            ##

            model_dict[rename] = model
            hist_dict[rename] = hist1
        
            y_pred = model.predict(X_test_t)
            rms = sqrt(mean_squared_error(y_test_v, y_pred))
            
            y_pred_lst.append(y_pred.reshape(-1)[0])
            rms_lst.append(rms)
            
        test_pred_df[name+'_y_pred'] = y_pred_lst
        test_pred_df[name+'_rms'] = rms_lst
        
    break



2016LG =======================================
0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 24 samples, validate on 1 samples
Epoch 1/100
24/24 [==============================] - 2s 92ms/sample - loss: 0.6188 - mae: 0.4546 - val_loss: 0.0705 - val_mae: 0.2655
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0020 - mae: 0.0373 - val_loss: 0.0200 - val_mae: 0.1413
Epoch 3/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0024 - mae: 0.0376 - val_loss: 0.0263 - val_mae: 0.1621
Epoch 4/100
24/24 [===

24/24 [==============================] - 2s 80ms/sample - loss: 0.7789 - mae: 0.5081 - val_loss: 0.0198 - val_mae: 0.1408
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0045 - mae: 0.0559 - val_loss: 0.0013 - val_mae: 0.0360
Epoch 3/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0036 - mae: 0.0446 - val_loss: 5.5681e-06 - val_mae: 0.0024
Epoch 4/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0034 - mae: 0.0415 - val_loss: 7.0798e-05 - val_mae: 0.0084
Epoch 5/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0032 - mae: 0.0387 - val_loss: 4.8116e-05 - val_mae: 0.0069
Epoch 00005: early stopping
7
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)       

24/24 [==============================] - 2s 92ms/sample - loss: 0.9253 - mae: 0.6197 - val_loss: 0.0068 - val_mae: 0.0825
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0059 - mae: 0.0574 - val_loss: 8.6446e-04 - val_mae: 0.0294
Epoch 3/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0038 - mae: 0.0466 - val_loss: 2.6178e-07 - val_mae: 5.1165e-04
Epoch 4/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0035 - mae: 0.0408 - val_loss: 5.9848e-05 - val_mae: 0.0077
Epoch 5/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0035 - mae: 0.0435 - val_loss: 9.2903e-04 - val_mae: 0.0305
Epoch 00005: early stopping
13
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dens

24/24 [==============================] - 0s 3ms/sample - loss: 0.0065 - mae: 0.0578 - val_loss: 0.0040 - val_mae: 0.0629
Epoch 3/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0032 - mae: 0.0371 - val_loss: 0.0022 - val_mae: 0.0468
Epoch 00003: early stopping
19
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 24 samples, validate on 1 samples
Epoch 1/100
24/24 [==============================] - 2s 70ms/sample - loss: 0.6830 - mae: 0.4658 - val_loss: 0.0211 - val_mae: 0.1451
Epoch 2/100
24/24 [==============================] -

24/24 [==============================] - 2s 72ms/sample - loss: 0.8129 - mae: 0.5056 - val_loss: 0.0141 - val_mae: 0.1186
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0076 - mae: 0.0636 - val_loss: 2.3691e-04 - val_mae: 0.0154
Epoch 3/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0037 - mae: 0.0493 - val_loss: 3.7931e-04 - val_mae: 0.0195
Epoch 4/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0055 - mae: 0.0555 - val_loss: 6.4057e-04 - val_mae: 0.0253
Epoch 00004: early stopping
2016HH =======================================
0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 6

...

24/24 [==============================] - 2s 81ms/sample - loss: 0.6941 - mae: 0.5530 - val_loss: 0.0305 - val_mae: 0.1746
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0169 - mae: 0.1100 - val_loss: 0.0048 - val_mae: 0.0696
Epoch 3/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0022 - mae: 0.0360 - val_loss: 1.4209e-04 - val_mae: 0.0119
Epoch 4/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0019 - mae: 0.0356 - val_loss: 2.1779e-05 - val_mae: 0.0047
Epoch 5/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0020 - mae: 0.0364 - val_loss: 1.7002e-04 - val_mae: 0.0130
Epoch 6/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0020 - mae: 0.0353 - val_loss: 1.0098e-04 - val_mae: 0.0100
Epoch 00006: early stopping
11
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)   

24/24 [==============================] - 2s 76ms/sample - loss: 0.8669 - mae: 0.5172 - val_loss: 0.0065 - val_mae: 0.0806
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0066 - mae: 0.0695 - val_loss: 5.7492e-05 - val_mae: 0.0076
Epoch 3/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0031 - mae: 0.0447 - val_loss: 0.0013 - val_mae: 0.0355
Epoch 4/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0020 - mae: 0.0362 - val_loss: 0.0037 - val_mae: 0.0608
Epoch 00004: early stopping
17
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
________________________

24/24 [==============================] - 2s 89ms/sample - loss: 0.8943 - mae: 0.5361 - val_loss: 0.0014 - val_mae: 0.0372
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0104 - mae: 0.0904 - val_loss: 1.7261e-04 - val_mae: 0.0131
Epoch 3/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0087 - mae: 0.0822 - val_loss: 2.7045e-04 - val_mae: 0.0164
Epoch 4/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0106 - mae: 0.0907 - val_loss: 1.8039e-05 - val_mae: 0.0042
Epoch 5/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0111 - mae: 0.0891 - val_loss: 0.0023 - val_mae: 0.0481
Epoch 6/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0075 - mae: 0.0725 - val_loss: 9.0892e-04 - val_mae: 0.0301
Epoch 00006: early stopping
3
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)    

24/24 [==============================] - 2s 93ms/sample - loss: 0.6097 - mae: 0.4993 - val_loss: 0.0513 - val_mae: 0.2265
Epoch 2/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0149 - mae: 0.1055 - val_loss: 0.0105 - val_mae: 0.1024
Epoch 3/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0091 - mae: 0.0837 - val_loss: 0.0297 - val_mae: 0.1724
Epoch 4/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0088 - mae: 0.0813 - val_loss: 0.0119 - val_mae: 0.1093
Epoch 00004: early stopping
9
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_____________________________

24/24 [==============================] - 2s 80ms/sample - loss: 0.9104 - mae: 0.5981 - val_loss: 0.0055 - val_mae: 0.0741
Epoch 2/100
24/24 [==============================] - 0s 4ms/sample - loss: 0.0216 - mae: 0.1202 - val_loss: 0.0091 - val_mae: 0.0954
Epoch 3/100
24/24 [==============================] - 0s 3ms/sample - loss: 0.0124 - mae: 0.0987 - val_loss: 0.0110 - val_mae: 0.1047
Epoch 00003: early stopping
21
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 24 samples, validate on 1 samples
Epoch 1/100
24/24 [==============================] -

In [9]:
test_pred_df.to_csv("2016_1fold_timestep3_PCT.csv",index=False)

In [15]:
tmp4

,T_ID,YEAR,PCT
0,HH,2016,0.500000
1,HT,2016,0.458333
2,KT,2016,0.291667
3,LG,2016,0.608696
4,LT,2016,0.500000
5,NC,2016,0.565217
6,OB,2016,0.666667
7,SK,2016,0.458333
8,SS,2016,0.500000
9,WO,2016,0.375000


In [17]:
tteam = []
for i in team:
    tteam.append(i)
    tteam.append(i)

In [28]:
mean_tdf = test_pred_df.mean().reset_index()

In [29]:
mean_tdf['T_ID'] = tteam

In [31]:
pd.merge(mean_tdf, tmp4, how='left')

,index,0,T_ID,YEAR,PCT
0,2016LG_y_pred,0.620275,LG,2016,0.608696
1,2016LG_rms,0.018753,LG,2016,0.608696
2,2016HH_y_pred,0.529412,HH,2016,0.500000
3,2016HH_rms,0.032717,HH,2016,0.500000
4,2016NC_y_pred,0.521020,NC,2016,0.565217
5,2016NC_rms,0.044198,NC,2016,0.565217
6,2016HT_y_pred,0.569602,HT,2016,0.458333
7,2016HT_rms,0.111269,HT,2016,0.458333
8,2016SK_y_pred,0.436350,SK,2016,0.458333
9,2016SK_rms,0.022064,SK,2016,0.458333


In [32]:
test_pred_df

,2016LG_y_pred,2016LG_rms,2016HH_y_pred,2016HH_rms,2016NC_y_pred,2016NC_rms,2016HT_y_pred,2016HT_rms,2016SK_y_pred,2016SK_rms,2016KT_y_pred,2016KT_rms,2016WO_y_pred,2016WO_rms,2016LT_y_pred,2016LT_rms,2016SS_y_pred,2016SS_rms,2016OB_y_pred,2016OB_rms
0,0.621868,0.013172,0.512575,0.012575,0.519706,0.045511,0.568578,0.110245,0.438210,0.020124,0.298377,0.006711,0.570998,0.195998,0.367136,0.132864,0.498755,0.001245,0.538121,0.128546
1,0.615497,0.006801,0.544843,0.044843,0.528023,0.037194,0.576373,0.118040,0.411534,0.046800,0.315917,0.024250,0.561132,0.186132,0.406828,0.093172,0.510076,0.010076,0.510485,0.156182
2,0.604509,0.004187,0.527245,0.027245,0.523910,0.041308,0.557155,0.098822,0.446607,0.011726,0.315391,0.023724,0.552193,0.177193,0.341503,0.158497,0.498428,0.001572,0.528300,0.138367
3,0.665460,0.056765,0.544449,0.044449,0.514071,0.051146,0.559257,0.100924,0.446979,0.011355,0.322750,0.031083,0.570387,0.195387,0.331594,0.168406,0.494898,0.005102,0.528865,0.137802
4,0.665083,0.056387,0.531441,0.031441,0.511310,0.053907,0.564598,0.106264,0.449006,0.009327,0.325778,0.034112,0.590735,0.215735,0.387885,0.112115,0.512520,0.012520,0.519864,0.146802
5,0.615320,0.006624,0.498892,0.001108,0.517489,0.047729,0.562366,0.104033,0.454492,0.003842,0.306605,0.014938,0.564129,0.189129,0.376083,0.123917,0.509552,0.009552,0.568688,0.097979
6,0.610347,0.001651,0.486579,0.013421,0.515104,0.050114,0.567467,0.109133,0.442656,0.015677,0.307724,0.016058,0.589634,0.214634,0.402342,0.097658,0.522531,0.022531,0.518869,0.147797
7,0.620401,0.011705,0.519247,0.019247,0.536586,0.028631,0.581000,0.122667,0.445056,0.013277,0.315767,0.024100,0.611390,0.236390,0.446740,0.053260,0.512752,0.012752,0.542284,0.124383
8,0.618127,0.009431,0.501691,0.001691,0.506140,0.059077,0.549291,0.090958,0.429074,0.029260,0.318122,0.026456,0.554459,0.179459,0.352851,0.147149,0.509103,0.009103,0.485954,0.180713
9,0.652242,0.043546,0.494868,0.005132,0.514441,0.050776,0.595484,0.137150,0.459337,0.001003,0.290624,0.001043,0.564477,0.189477,0.399936,0.100064,0.499436,0.000564,0.512781,0.153886


In [12]:
y_test

,PCT
3,0.608696


##### TABLE

||2016HH_ypred|2016_HH_y| ....|
|-|-|-|-|
|model1| 0.xxx|0.yyy |... |
|model2| 0.xxx|0.yyy |... |
|...| ...|.... |... |
|modelNN| 0.xxx|0.yyy |... |

In [ ]:
X_train